* [Go to input parameters](#inputParameters)

For more details about meanings of ids go to [this link](https://docs.google.com/spreadsheets/d/1uZLtoB3OijjfMOUJHBkoL542wfa4eLv_JgjrT5e1oGI/edit#gid=577660682)

In [1]:
import pandas as pd
import numpy as np
import os 
from os import listdir, remove
from os.path import isfile, join, exists


# -------------------------------------------------------

def ls(folder_path):
    """
    ls() list all the files in the input folder
    """
    files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    files.sort()
    return files


# -------------------------------------------------------

def saveFile(path, data, labels = '', fmt = '%.18e'):
    '''
    function to save data into either new file or appending to file.
    path: path to file
    data: either np.ndarray or pd.dataframe
    labels: use when data is numpy. When data is dataframe, labels are the names from input data
    fmt: use only when data is numpy
    '''
    if type(data) is np.ndarray:
        if not os.path.isfile( path ):
            # create new file        
            np.savetxt( path , data , delimiter = ';', header = labels, comments = '', fmt = fmt)
        else:
            # append to file
            with open(path, "a") as f:
                np.savetxt(f , data , delimiter = ';', comments = '', fmt = fmt )
    elif isinstance(data, pd.DataFrame):
        if not os.path.isfile( path ):
            # create new file        
            data.to_csv(path, na_rep = 'None', sep = ';', index = False)
        else:
            # append to file
            data.to_csv(path, na_rep = 'None', sep = ';', index = False, mode='a', header=False)

## General Data:
- It computes stimuli perception, number of screen touches and test time
- Future work to add: general data as phone type, starting time, etc.

In [2]:
def compute_stimuli_perpception():
    '''
    ## Stimuli perception:
    it takes out users' answers from stimuli perception such as audio and vibration
    '''
    
    output_file = 'stimuli_perception.csv'

    stimuli_A = df[ df['id_msg'] == 33 ]
    stimuli_H = df[ df['id_msg'] == 34 ]
    volume = df[ df['msg'] == ' volume (%) ' ]['value'].values[0]

    if len(stimuli_A)!=1 or len(stimuli_H)!=1:
        print('Error in stimuli')
    else:
        # concatenate values
        stimuli = np.reshape( [ stimuli_A['value'].to_numpy(dtype = str) , stimuli_H['value'].to_numpy(dtype=str) ] , (1, 2) )
        # add type-test, id-user and volume
        # stimuli = np.append ( [[type_test, id_user, volume]] , stimuli, axis = 1) # OLD version
        stimuli = np.append ( [[type_test, volume]] , stimuli, axis = 1)
        # test if test type and self perception is okay
        perception = 'FALSE'
        if stimuli[0][0] == ' Base'      and stimuli[0][2] == ' no'  and stimuli[0][3] == ' no':
            perception = 'TRUE'
        elif stimuli[0][0] == ' Auditory' and stimuli[0][2] == ' yes'  and stimuli[0][3] == ' no':
            perception = 'TRUE'
        elif stimuli[0][0] == ' Haptic'   and stimuli[0][2] == ' no'  and stimuli[0][3] == ' yes':
            perception = 'TRUE'
        elif stimuli[0][0] == ' HapticAuditory' and stimuli[0][2] == ' yes'  and stimuli[0][3] == ' yes':
            perception = 'TRUE'
    
        df_stimuli = pd.DataFrame( {
                         'type_test' : [stimuli[0][0]] ,
                         'volume'    : [stimuli[0][1]] ,
                         'Auditory'  : [stimuli[0][2]] ,
                         'Haptic'    : [stimuli[0][3]] ,
                         'Perception': perception,
                        }
                      )
        return df_stimuli
        

def computing_nScreenTouch():
    output_file_touches = 'touches_data.csv'
    # selecting relevant log messages
    touches_data = df.loc[ df['id_msg'].isin( [1, 5, 10, 11, 21] ) ]
    old_touch = ''
    touch_counter = 0
    touches = []
    for ix, touch in touches_data.iterrows():
        if touch['id_msg'] == 1: # log for new level
            # print(touch['id_msg'], touch['msg'], touch['value'])
            ;
        elif touch['id_msg'] == 10: # log for new ball position by system
            # print(touch['id_msg'], touch['msg'], touch['value'])
            touch_counter = 0
            old_touch = ''
        elif touch['id_msg'] == 11: # log for new ball position by user
            # print('55', 'screen touches', touch_counter)
            touches.append(touch_counter)
            # print(touch['id_msg'], touch['msg'], touch['value'])
            touch_counter = 0
            old_touch = ''
        elif touch['id_msg'] == 21: # log for screen of pattern to answer
            counter_running = False # stop counter and end without adding results
        elif touch['id_msg'] == 5:
            if touch['value'] != old_touch:
                touch_counter += 1
                old_touch = touch['value']
    #
    df_touches_results = pd.DataFrame( {
                             'touches_1' : [touches.count( 1 )],
                             'touches_2' : [touches.count( 2 )],
                             'touches_3' : [touches.count( 3 )],
                             'touches_4' : [touches.count( 4 )],
                             'touches_5+': touches.count( sum(i > 5 for i in touches) ),
                            }
                          )
    return df_touches_results
    
    

def compute_general_data():
    '''
    it computes general info from user and type of test. 
    Reslts are save into out file users_general_data.csv 
    '''
    
    output_file_general = 'users_general_data.csv'

    #getting general data
    test_time = df[ df['msg']==' app starts ' ].iloc[1]['value'].split(' ')[2]
    age   = df[ df['msg'] == ' age ']['value'].values[0]
    work  = df[ df['msg'] == ' work ']['value'].values[0]
    sex   = df[ df['msg'] == ' sex ']['value'].values[0]
    hand  = df[ df['msg'] == ' hand dominant ']['value'].values[0]
    mobile= df[ df['msg'] == ' mobile type ']['value'].values[0]
    df_stimuli = compute_stimuli_perpception()
    df_touches_results = computing_nScreenTouch()

    # composing output dataframe
    df_general = pd.DataFrame( {'id_user'   : [id_user], 
                                'test_time' : [test_time],
                                'age'       : [age],
                                'work'      : [work],
                                'sex'       : [sex],
                                'hand'      : [hand],
                                'mobile'    : [mobile],
                               })
    df_general = pd.concat([df_general, df_touches_results, df_stimuli], axis=1)

    saveFile(output_folder + output_file_general, df_general)

## Nasa data:
it takes out data from the nasa tests

In [3]:
def compute_nasa():
    output_file = 'nasa_stats.csv'

    nasa = df[ df['id_msg']==32 ]

    if len( nasa ) < 18:

        print('Error in nasa. User did not finisht the test!')
    
    elif len( nasa ) > 18:

        print('Error in nasa. User did the test more than one time!')

    else:

        label_cols = 'type_test; id_user; level; Mental; Physic; Temporal; Performance; Effort; Frustration; Total'

        # low level
        level = 0
        # reshape data
        nasa_low = nasa[0:6].pivot(index = 'id_msg', columns="value", values="value_extra").to_numpy(dtype = float)
        # removing physical demand to compute and add total nasa
        nasa_low = np.append ( nasa_low, [[ np.sum( nasa_low[0, [0, 2, 3, 4, 5]] ) ]] , axis = 1)
        # truncate decimals
        nasa_low = np.around(nasa_low, 3)
        # add type-test, id-user and level
        nasa_low = np.append ( [[type_test, id_user, level]] , nasa_low, axis = 1)
        # saving file
        saveFile( output_folder + output_file ,nasa_low , label_cols , fmt='%s' )

        # med level
        level = 1
        nasa_med = nasa[6:12].pivot(index = 'id_msg', columns="value", values="value_extra").to_numpy(dtype = float)
        nasa_med = np.append ( nasa_med, [[ np.sum( nasa_med[0, [0, 2, 3, 4, 5]] ) ]] , axis = 1)
        nasa_med = np.around(nasa_med, 3)
        nasa_med = np.append ( [[type_test, id_user, level]] , nasa_med, axis = 1)
        saveFile( output_folder + output_file ,nasa_med , label_cols , fmt='%s' )

        # high level
        level = 2
        nasa_high = nasa[12:18].pivot(index = 'id_msg', columns="value", values="value_extra").to_numpy(dtype = float)
        nasa_high = np.append ( nasa_high, [[ np.sum( nasa_high[0, [0, 2, 3, 4, 5]] ) ]] , axis = 1)
        nasa_high = np.around(nasa_high, 3)
        nasa_high = np.append ( [[type_test, id_user, level]] , nasa_high, axis = 1)
        saveFile( output_folder + output_file , nasa_high , label_cols ,fmt='%s' )

## Gaming score:
it analyses data from balls and patterns

In [4]:
def analyze_ball_data(df_level, level, nBallsInLoop):        
    ball_moves = df_level[ df_level['id_msg'].isin([10,11,24]) ] # dataframe including movement balls by system (10) and by user (11)
        
    test = []
    user = []
    level_task_1 = []
    system_ball_task_1 = []
    success_task_1 = []
    time_task_1 = []
    reaction_time_task_1 = []
    kBallsInLoop = nBallsInLoop
    nValidBalls = 0
    first_ball_pattern = True # flag indicating this is the first ball in a pattern loop
    
    for idx, ball in ball_moves.iterrows():
        type_move = ball['id_msg']
        if type_move == 24:
            # new pattern loop
            kBallsInLoop = nBallsInLoop
            first_ball_pattern = True
        else: 
            ball_time = ball['time']
            value     = int( ball['value'] )
            #
            if type_move == 10: # new ball position by system
                                                                                   
                if not first_ball_pattern: # if it is not the first system ball, it is allowed to save
                    # if expected_moves != 0:
                    if kBallsInLoop < 0:
                        # here means valid system balls were registered. Sometimes more balls can appear because delays in the app. But they are not valid
                        continue
                    #
                    # OUTPUT VALUES:
                    #
                    nValidBalls += 1
                    test.append( type_test )
                    user.append( id_user )
                    level_task_1.append( level )                    
                    system_ball_task_1.append( expected_moves )
                    # if nBallsInLoop-kBallsInLoop == 1:
                    #    print('>>{} : {} : {}'.format( nBallsInLoop-kBallsInLoop, start_time , expected_moves )) # todo: remove line
                    # else: 
                    #     print('>{} : {} : {}'.format( nBallsInLoop-kBallsInLoop, start_time , expected_moves )) # todo: remove line
                    #
                    if num_moves - abs(expected_moves) == 0 and user_position == 0: # success
                        # success_task_1[k_ball] = True
                        success_task_1.append( True )
                        time_task_1.append( user_time - start_time )
                    else: # not success
                        success_task_1.append( False )
                        time_task_1.append( None )
                    if num_moves > 0:
                        reaction_time_task_1.append( reaction_time )
                    else: # user never moved the ball
                        reaction_time_task_1.append( None )
                #
                # set values of variables to start analysis for current system ball position
                first_ball_pattern = False
                expected_moves = value
                start_time =  ball_time
                num_moves  = 0
                user_time  = 0
                first_user_move = True
                #
                kBallsInLoop -= 1

            elif type_move == 11: # new ball position by user
                num_moves += 1 # counting the user movements after each time system moved the ball
                user_time = ball_time
                user_position = value
                if first_user_move:            
                    first_user_move = False
                    reaction_time = user_time - start_time
    #
    dataset = pd.DataFrame( {'type_test': test,
                             'id_user'  : id_user,
                             'level'    : level_task_1, 
                             'system_ball': system_ball_task_1,
                             'success'  : success_task_1,
                             'time_ms'  : time_task_1,
                             'reaction_time_ms' : reaction_time_task_1
                            }
                          )
    
    return dataset, nValidBalls

# -------------------------------------------------------

def analyze_pattern_data(df_level, level):
    patterns_data = df_level.loc[ df_level['id_msg'].isin([20,21,23,24]) ]
    # pattern by system
    expected_pattern    = patterns_data[ patterns_data['id_msg'] == 20 ]['value_extra']
    expected_pattern_id = patterns_data[ patterns_data['id_msg'] == 20 ]['value']
    # pattern by user
    answered_pattern = patterns_data[ patterns_data['id_msg'] == 24 ]['value_extra']
    # level
    n_patterns = len( answered_pattern )
    level_task_2 = level * np.ones( n_patterns, dtype = np.int32 )
    # type-test
    test = [type_test]*n_patterns
    # id user
    user = id_user * np.ones( n_patterns, dtype = np.int32 )
    # user time
    start_time = patterns_data[ patterns_data['id_msg'] == 21 ]['time'].astype(np.float)
    end_time   = patterns_data[ patterns_data['id_msg'] == 23 ]['time'].astype(np.float)
    if len(start_time) != len(end_time):   
        # when user does not makes OK in answer-patterns
        #
        end_time_syst = patterns_data[ patterns_data['id_msg'] == 24 ]['time'].astype(np.float)
        #
        if len(end_time) == 0:
            # when user never does OK and system automatically goes to next pattern
            total_time = end_time_syst.values - start_time.values
        else:
            # when user does not OK by some answer patterns
            total_time = np.zeros( n_patterns )
            ix_start = 0
            ix_end = 0
            for tm in end_time_syst:
                if not ( min( np.abs( tm - end_time ) ) < 2000 ):
                    # print("no user answer at pattern: {}".format(ix_start))
                    total_time[ix_start] = tm - start_time.values[ix_start]
                    ix_start += 1        
                else:
                    total_time[ix_start] = end_time.values[ix_end] - start_time.values[ix_start]
                    ix_start += 1
                    ix_end   += 1
    else:
        # when user does OK by all answer patterns
        total_time = end_time.values - start_time.values
    #
    success = np.zeros( n_patterns, dtype=bool )
    for k in range( n_patterns ):
        success[k] = answered_pattern.iloc[k] == expected_pattern.iloc[k]
        if success[k] == False:
            total_time[k] = np.nan # None
    #
    dataset = pd.DataFrame( {'type_test' : test,
                             'id_user'   : user,
                             'level'     : level_task_2,                              
                             'success'   : success,
                             'total_time_ms': total_time,
                             'expected_pattern_id': expected_pattern_id.values,
                             'expected_pattern'   : expected_pattern.values,                         
                             'answered_pattern'   : answered_pattern.values,
                            }
                          )
    return dataset

In [5]:
def compute_gaming_score():
    output_file_ball = 'ball_data.csv'
    output_file_pattern = 'pattern_data.csv'

    start_level = df[ df['id_msg'] == 1 ].astype({'value': 'int32'})
    end_level   = df[ df['id_msg'] == 2 ].astype({'value': 'int32'})
    # number of ball by each pattern loop
    nBallsInLoop = 8
    expectedBalls = (nBallsInLoop*6 , nBallsInLoop*8 , nBallsInLoop*9)
    errors = 0 # count number of errors
    if len(start_level)!=3 or len(end_level)!=3:
        print('Error. user did the game more than one time or exit before ending')
    else:
        success_task_1 = np.zeros(3)
        total_task_1 = np.zeros(3)
        df_ball_results = [None] * 3
        df_patterns_results = [None] * 3
        for level in range(3):
            # slicing data by level
            s = (start_level[ start_level['value'] == level] ).index.values
            e = (end_level[ end_level['value'] == level] ).index.values
            df_level = df[s[0]:e[0]+1]
            # get_valid_ball_data removes balls in zero and system movements of the ball small than 3 secs
            df_ball_results[level] , numValidBalls = analyze_ball_data(df_level, level, nBallsInLoop)
            # analyse patterns data
            df_patterns_results[level] = analyze_pattern_data(df_level, level)
            #
            if numValidBalls == expectedBalls[level]:
                print('level: {} - {}balls : OK'.format(level , numValidBalls))            
            else:
                print('level: {} - {}balls : ERROR'.format(level , numValidBalls))
                errors += 1
        #    
        # appending results to one dataframe
        df_ball_results = df_ball_results[0].append( df_ball_results[1] ).append( df_ball_results[2] )
        df_patterns_results = df_patterns_results[0].append( df_patterns_results[1] ).append( df_patterns_results[2] )

        # adding a column as id for each row
        rowID = np.arange( len(df_ball_results) )
        df_ball_results.insert(0, 'id', rowID, allow_duplicates = False)
        rowID = np.arange( len(df_patterns_results) )
        df_patterns_results.insert(0, 'id', rowID, allow_duplicates = False)
        
        # save results
        saveFile(output_folder + output_file_ball, df_ball_results)
        saveFile(output_folder + output_file_pattern, df_patterns_results)
    #
    return errors

# Pole position:

In [6]:
def str2logic(val):
    '''
    transform boolean values of pattern shapes to numeric values
    '''
    if val == 'True' or val == ' True':
        return 1
    else: 
        return -1

def compute_scores(levels, out_file_score):
    '''
    levels could be [0,1] or [2]
    '''
    in_folder = output_folder
    in_file_ball = 'ball_data.csv'
    in_file_pattern = 'pattern_data.csv'    
    
    df_ball    = pd.read_csv(in_folder+in_file_ball, sep = ';')
    df_pattern = pd.read_csv(in_folder+in_file_pattern, sep = ';')

    #users = pd.unique(df_ball['id_user']) # it uses all users in ball_file and pattern_file (old and new ones)
    users = new_users # variable taken from cell of input_parameters
    ball_time_ms = 2250 # fixed time in app to update ball position by system
    nPerfectBalls = [] # number of perferct ball matches
    nTotalBalls = []
    nPerfectPatterns = [] # number of perfect pattern matches
    nTotalPatterns = []
    score_ball = [] # [0,1] the most spent time, the less value. It is a subtraction between the total available time and the user's spent time
    score_pattern = [] # [0,1] score as approximation to the pattern recall. +1 for good cells, -0.5 for wrong cells
    score_total = [] # [0,1] average value between score_ball and score_pattern
    score_perfect = [] # [0,1] average between scores for perfect ball match and perfect pattern match
    type_test = []
    for id_user in users:
        # ball analysis
        df_user = df_ball[ df_ball['id_user']==id_user ]
        df_user = df_user[ df_user['level'].isin(levels) ] # select only levels 0 and 1
        type_test.append( df_user['type_test'].values[0] )
        perfectBalls = df_user['success'].sum()
        nPerfectBalls.append( perfectBalls ) # balls moved to center with minimal movements
        totalBalls = len(df_user)
        nTotalBalls.append(totalBalls)
        spent_time = pd.to_numeric( df_user['time_ms'] , errors = 'coerce').fillna(ball_time_ms).sum()
        score_ball.append( (ball_time_ms*totalBalls - spent_time)/(ball_time_ms*totalBalls) ) # ball score
                
        #pattern analysis
        df_user = df_pattern[ df_pattern['id_user']==id_user ]
        df_user = df_user[ df_user['level'].isin(levels) ] # select only levels 0 and 1
        expected_good_cells = 0
        wrong_cells = 0
        good_cells = 0
        for ix, data_pattern in df_user.iterrows():
            expected = list( map( str2logic , data_pattern['expected_pattern'].split(',') ) )
            answered = list( map( str2logic , data_pattern['answered_pattern'].split(',') ) )

            expected_good_cells += np.sum( len(expected) )

            total_cells = len(expected)
            errors = np.array(expected) - np.array(answered)
            wrong_cells += np.sum(errors != 0) # cumulative of wrong cells by pattern
            good_cells  += total_cells - wrong_cells # cumulative of good cells by pattern

        score_pattern.append( (good_cells - 0.5*wrong_cells)/expected_good_cells ) # pattern ball
        perfectPatterns = df_user['success'].sum()
        totalPatterns = len(df_user)
        nPerfectPatterns.append( perfectPatterns ) # perfect match in answered-patterns
        nTotalPatterns.append( totalPatterns ) # total shown patterns
        score_perfect.append( 0.5*( perfectBalls/totalBalls + perfectPatterns/totalPatterns ) )

    #
    # total score: 50% ball score + 50% pattern score
    score_total = 0.5*np.add(score_ball , score_pattern)

    df_score = pd.DataFrame( {'id_user'       : users,
                              'type_test'     : type_test,
                              'num_perfect_balls': nPerfectBalls,
                              'total_balls'   : nTotalBalls,
                              'num_perfect_patterns': nPerfectPatterns,
                              'total_patterns': nTotalPatterns,
                              'perfect_score' : score_perfect,
                              'score_ball'    : score_ball,                              
                              'score_pattern' : score_pattern,
                              'score_total'   : score_total,
                            }
                          )

    # save score file
    saveFile(output_folder + out_file_score, df_score)
    print('scores computed for {} new users'.format(len(users)))
    # print(users)
    # print(score_ball)
    # print(score_pattern)
    # print(score_total)

### Input parameters: <a class="anchor" id="inputParameters"></a>
Name format for input files is: log_##.txt, where ## hast to be an integer number

In [7]:
# ------------------------- INPUT PARAMETERS ------------------------------------

# run analysis
input_foldername = './../../../Data/Data_cogito/Raw_data/final_tests/2_filter2/'
output_folder    = './../../../Data/Data_cogito/Processed_data/final_tests/2_filter2/'
input_files = ['all'] # ['Log_02.txt'] , ['all']

# -------------------------------------------------------------------------------

In [8]:
if input_files[0] == 'all':
    files = ls(input_foldername)
else:
    files = input_files
#
nUsers = 0 # number of registered users
nGames = 0 # number of registered games
nErrors = 0
new_users = [] # list of new users added to be analyzed
for file in files:
    if "readme" in file:
        continue
    print(file)
    id_user = int(file.split('.')[0].split('_')[1])
    new_users.append(id_user)
    col_names = [ 'time',    'id_msg',    'msg',    'value',    'value_extra' ]
    df = pd.read_csv(input_foldername + file , header = None, sep = '_', na_values = 'NA', names = col_names)
    # type test
    type_test = df[ df['msg'] == ' test version ' ]['value'].values[0]

    # number of registered games
    nTimesPlayed = len( df[ (df['id_msg']==2) & (df['value']==' 0') ] )
    nUsers += 1
    nGames += nTimesPlayed
    if nTimesPlayed == 1:
        print( 'nGames = {} : OK'.format(nTimesPlayed) )
    else:
        print( 'nGames = {} : ERROR'.format(nTimesPlayed) )
        nErrors += 1
    #
    # compute general data: stimuli perception, screen touches and test time
    compute_general_data()
    # compute relevant information: nasa and tasks results
    compute_nasa()
    nErrors += compute_gaming_score()
    print('')

if (nErrors != 0):
    print('ERROR! see above for details')
else:
    print("Everything seems to be: OK")
print('ready')

Log_01.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_012.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_013.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_014.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_015.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_017.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_018.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_02.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_020.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72balls : OK

Log_021.txt
nGames = 1 : OK
level: 0 - 48balls : OK
level: 1 - 64balls : OK
level: 2 - 72ball

**POST ANALYSIS:**

Copy these results to the excel file to the corresponding table

- table for gamers score: it will work only with level 0 and 1

In [9]:
# compute_scores( levels = [0,1] , out_file_score = 'users_score_lv0-lv1.csv')
compute_scores( levels = [0] ,   out_file_score = 'users_score_lv0.csv')
compute_scores( levels = [1] ,   out_file_score = 'users_score_lv1.csv')
compute_scores( levels = [2] ,   out_file_score = 'users_score_lv2.csv')

scores computed for 66 new users
scores computed for 66 new users
scores computed for 66 new users


- to know how many users per type of test:

In [10]:
in_file = 'ball_data.csv'
# in_folder = './../../../Data/Data_cogito/Processed_data/'
in_folder = output_folder

df = pd.read_csv(in_folder+in_file, sep = ';')
nB = len( df[df['type_test'] == ' Base']['id_user'].unique() )
nH = len( df[df['type_test'] == ' Haptic']['id_user'].unique() )
nA = len( df[df['type_test'] == ' Auditory']['id_user'].unique() )
nHA = len( df[df['type_test'] == ' HapticAuditory']['id_user'].unique() )

print('Base users: \t{}'.format(nB))
print('Haptic users: \t{}'.format(nH))
print('Auditory users: {}'.format(nA))
print('HapticAuditory users: {}'.format(nHA))
print('\nTotal users: \t{}'.format( nHA + nH + nA + nB ) )

Base users: 	15
Haptic users: 	17
Auditory users: 16
HapticAuditory users: 18

Total users: 	66
